In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
%matplotlib inline

In [2]:
cd ..\experiments\data\human\1.7

C:\Users\piichan\Documents\Berkeley\CoCoSci Lab\mcrl\mcrl\experiments\data\human\1.7


In [3]:
df=pd.read_csv('trials.csv', sep=',')

In [4]:
import ast
toList = lambda x: ast.literal_eval(x)
df['clicks'] = df['clicks'].apply(toList)

In [5]:
df.info_cost.unique()

array([ 0.1 ,  1.25,  4.  ])

In [6]:
df.sort_values('info_cost')
low_cost = df.loc[df.info_cost == 0.1]
med_cost = df.loc[df.info_cost == 1.25]
high_cost = df.loc[df.info_cost == 4]

In [7]:
count = 0
for seq in df['clicks']:
    if seq == []:
        count += 1
        
count/df['clicks'].size

0.30399568034557234

In [8]:
cost_type = low_cost
brfs_count = 0
dfs_count = 0
top = set([9, 10, 11, 12])
left = set([13, 14, 15, 16])
right = set([5,6,7,8])
bottom = set([1, 2, 3, 4])
for seq in cost_type['clicks']:
    if seq == []:
        continue     
    if len(seq) < 4:
        continue
    first_four = seq[:4]
    if set(first_four) == set([1,5,9,13]):
        if len(seq) > 4:
            rest = seq[4:]
            if len(rest) > 4:
                rest = seq[4:8]
            if set(rest) <= set([2,6,10,14]):
                brfs_count += 1 
        else:
            brfs_count += 1
    elif (set(first_four) == top)or(set(first_four) == right)or(set(first_four) == bottom)or(set(first_four) == left):
        if len(seq) > 4:
            rest = set(seq[4:])
            more = False
            if len(rest) > 4:
                more = True
                rest = set(seq[4:8])
            if (rest<=top)or(rest<=right)or(rest<=left)or(rest<=bottom):
                if more:
                    rest = set(seq[8:])
                    if len(rest) > 4:
                        rest = set(seq[8:12])
                    if (rest<=top)or(rest<=right)or(rest<=left)or(rest<=bottom):
                            dfs_count += 1                    
                else:
                    dfs_count += 1
        else:
            dfs_count += 1
    else: #between 1 and 3 clicks were made
        if (set(seq) < top)or(set(seq) < right)or(set(seq)< bottom) or (set(seq) < left):
            dfs_count += 1
        elif set(seq) < set([1, 5, 9, 13]):
            brfs_count += 1
        
(brfs_count/cost_type['clicks'].size) * 100

5.0245098039215685

ver 0.991 

values are given in percentages; top to bottom is low, med, and high cost

i stands for initial (only the first 4 clicks) while f stands for full (entire sequence)

| Breadth-First | Depth-First(i)| Depth-First(f)|
| ------------- |:-------------:| -------------:|
| 3.02          | 12.50         |8.47       |
| 6.07          | 12.13         |    9.375      |
| 4.23          | 3.43          |    2.42       |



In [2]:
def depth_first(state, cur_clicks, dist, prune_t):
    prob_next_click = np.zeros(17)
    observed_states = np.nonzero(state)[0] #works only because indices represent location
    possible_clicks = []
    last_click = cur_clicks[:-1]
    pruned_nodes = prune(prune_t, state)
    if len(last_click) == 0:
        possible_clicks += [1, 5, 9, 13]
    elif last_click in [1, 5, 9, 13]:
        if last_click+1 in observed_states:
            for click in [last_click+2, last_click+3]:
                if (click not in observed_states) and (click not in pruned_nodes):
                    possible_clicks.append(click)
            if len(possible_clicks) == 0:
                for click in [1, 5, 9, 13]:
                    if (click not in observed_states) and (click not in pruned_nodes):
                        possible_clicks.append(click)
        else:
            if last_click+1 not in pruned_nodes:
                possible_clicks.append(last_click+1)
            else:
                click_candidates = [1, 5, 9, 13].remove(last_click)
                for c in click_candidates:
                    if (c in observed_states) or (c in pruned_nodes):
                        click_candidates.remove(c)
                possible_clicks += click_candidates
    else:
        if last_click+1 in observed_states:
            if last_click+2 in observed_states:
                for click in [1, 5, 9, 13]:
                    if (click not in observed_states) and (click not in pruned_nodes):
                        possible_clicks.append(click)
            elif (last_click+2 < 17) and (last_click+2 not in pruned_nodes):
                possible_clicks.append(last_click+2)
            else:
                for click in [1, 5, 9, 13]:
                    if (click not in observed_states) and (click not in pruned_nodes):
                        possible_clicks.append(click)
        elif (last_click+1 < 17) and (last_click+1 not in pruned_nodes):
            possible_clicks.append(last_click+1)
        else:
            for click in [1, 5, 9, 13]:
                    if (click not in observed_states) and (click not in pruned_nodes):
                        possible_clicks.append(click)
    if len(possible_clicks) != 0:
        for possibility in possible_clicks:
            if dist == 'Uniform':
                prob_next_click[possibility] = 1/len(possible_clicks)
    return prob_next_click

In [31]:
test_state = [0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1]
res = depth_first(test_state, 16, 'Uniform')
res

array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,
        0. ,  0. ,  0.5,  0. ,  0. ,  0. ])

In [3]:
#needs to be tested
def breadth_first(state, cur_clicks, dist, prune_t):
    prob_next_click = np.zeros(17)
    observed_states = np.nonzero(state)[0]
    possible_clicks = []
    last_click = cur_clicks[:-1]
    pruned_nodes = prune(prune_t, state)
    if len(last_click) == 0:
        possible_clicks += [1, 5, 9, 13]
    elif last_click in [1, 5, 9, 13]:
        if set([1, 5, 9, 13]).issubset(set(observed_states)):
            if set([2, 6, 10, 14]).issubset(set(observed_states)):
                for click in [3, 4, 7, 8, 11, 12, 15, 16]:
                    if (click not in observed_states) and (click not in pruned_nodes):
                        possible_clicks.append(click)
            else:
                for click in [2, 6, 10, 14]:
                    if (click not in observed_states) and (click not in pruned_nodes):
                            possible_clicks.append(click)
        else:
            for click in [1, 5, 9, 13]:
                if (click not in observed_states) and (click not in pruned_nodes):
                        possible_clicks.append(click)
    elif last_click in [2, 6, 10, 14]:
        if set([2, 6, 10, 14]).issubset(set(observed_states)):
            for click in [3, 4, 7, 8, 11, 12, 15, 16]:
                if (click not in observed_states) and (click not in pruned_nodes):
                    possible_clicks.append(click)
        else:
            for click in [2, 6, 10, 14]:
                if (click not in observed_states) and (click not in pruned_nodes):
                    possible_clicks.append(click)
    else:
        for click in [3, 4, 7, 8, 11, 12, 15, 16]:
                if (click not in observed_states) and (click not in pruned_nodes):
                    possible_clicks.append(click)
    if len(possible_clicks) != 0:
        for possibility in possible_clicks:
            if dist == 'Uniform':
                prob_next_click[possibility] = 1/len(possible_clicks)
    return prob_next_click

[1, 3]

In [4]:
#assumes that average reward value is 0
#needs to be tested
def best_first(state, cur_clicks, dist, prune_t):
    prob_next_click = np.zeros(17)
    observed_states = np.nonzero(state)[0]
    possible_clicks = []
    last_click = cur_clicks[:-1]
    pruned_nodes = prune(prune_t, state)
    if len(last_click) == 0:
        possible_clicks += [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
    else:
        for i in np.arange(1,17):
            if (i not in observed_states) and (i not in pruned_nodes):
                possible_clicks.append(i)
    if len(possible_clicks) != 0:
        for possibility in possible_clicks:
            if possibility in [1, 5, 9, 13]:
                indices = [possibility+1, possibility+2]
                path_1 = np.sum(state[indices])
                indices[1] += 1
                path_2 = np.sum(state[indices])
                prob_next_click[possibility] = np.max(path_1, path_2)
            elif possibility in [2, 6, 10, 14]:
                indices = [possibility-1, possibility+1]
                path_1 = np.sum(state[indices])
                indices[1] += 1
                path_2 = np.sum(state[indices])
                prob_next_click[possibility] = np.max(path_1, path_2)
            elif possibility in [4, 8, 12, 16]:
                indices = [possibility-1, possibility-3]
                prob_next_click[possibility] = np.sum(state[indices])
            else:
                indices = [possibility-1, possibility-2]
                prob_next_click[possibility] = np.sum(state[indices])
        sum_of_paths = np.sum(prob_next_click)
        for j in np.arange(len(prob_next_click)):
            path_val = prob_next_click[j]
            if path_val != 0:
                prob_next_click[j] = np.exp(path_val)/np.exp(sum_of_paths) #softmax function                
    return prob_next_click

In [35]:
state = np.array([0, 0, 20, 0, 0, 12, -8, 0, 0, -4, 0, 0, 0, 10, 0, 0, 0])
observed_states = np.nonzero(state)[0]
indices = [2, 11, 6]
indices[2] += 1
state[indices]

array([20,  0,  0])

In [5]:
def prune(threshold, state):
    #returns nodes on pruned branches
    pruned = []
    observed_states = np.nonzero(state)[0]
    bottom = [1, 2, 3, 4]
    top = [9, 10, 11, 12]
    right = [5, 6, 7, 8]
    left = [13, 14, 15, 16]
    direction = False
    if len(observed_states) != 0:
        for loc in observed_states:
            if (loc in [1, 2]) and (loc not in pruned):
                direction = bottom
            elif (loc in [5, 6]) and (loc not in pruned):
                direction = right
            elif (loc in [9, 10]) and (loc not in pruned):
                direction = top
            elif (loc in [13, 14]) and (loc not in pruned):
                direction = left
            if direction:
                path_val = np.sum(state[direction])
                if path_val < threshold:
                    pruned += direction         
    return pruned        

In [6]:
def satisficing_stop(state, threshold):
    bottom = [1, 2, 3, 4]
    top = [9, 10, 11, 12]
    right = [5, 6, 7, 8]
    left = [13, 14, 15, 16]
    for direction in [bottom, top, right, left]:
        observed_value = np.sum(state[direction])
        if observed_value > threshold:
            return True
    return False    

In [7]:
def strategy_likelihood(click_seq, strategy, cost_per_click, rewards, p_error, 
                        log=False, dist='Uniform', prune_t=-30, stop=(0,0)):
    #Stop should be a tuple of (stopping rule type, threshold)
    prob = 0
    cur_state = np.zeros(17)
    cur_clicks = []
    stop_type = stop[0]
    stop_threshold = stop[1]
    for click in click_seq:
        cur_clicks.append(click)
        possible_next_clicks = strategy(cur_state, cur_clicks, dist, prune_t)
        next_state = np.copy(cur_state)
        next_state[click] = rewards[click]
        last_click = click
        cur_state = next_state
        n_possible_clicks = np.count_nonzero(possible_next_clicks)
        p_strat = possible_next_clicks[click] 
        if stop_type == 1: # number of clicks x cost per click
            if (len(cur_clicks)*cost_per_click) > stop_threshold:
                possible_next_clicks = np.zeros(17)
        elif stop_type == 2: #satisficing
            stop_flag = satisficing_stop(cur_state, stop_threshold)
            if stop_flag:
                possible_next_clicks = np.zeros(17)
        num_wrong_clicks = 16 - (np.count_nonzero(cur_state)+n_possible_clicks)
        if dist == 'Uniform':
            p_click_given_state = (((1-p_error)*p_strat) + (p_error/num_wrong_clicks))
        if log:
            prob += np.log(p_click_given_state)
        else:
            prob *= p_click_given_state
    return prob